In [1]:
# 03 FX Signal Execution — Risk Reversals via IB
# ==================================================
# Detect signals, score quality, size positions, execute via IB.
# Budget: $5,000 (scale up as confidence grows).

import sys, warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

project_root = Path.cwd().parent if Path.cwd().name == 'signals' else Path.cwd()
sys.path.insert(0, str(project_root / 'signals' / 'scripts'))
sys.path.insert(0, str(project_root / 'src'))

from fx_signal_execute import (
    detect_active_signals, score_signal_quality,
    compute_position_size, construct_risk_reversal,
    execute_fx_signals, check_fx_positions, close_expired_signals,
    find_25delta_strikes,
    MARGIN_PER_CONTRACT, SIZING_MAP, CME_FX_CONFIG,
)

DATA_DIR = Path.home() / 'trade_data' / 'ETFTrader'
LIVE_DIR = project_root / 'fx_signals_live'
TOTAL_BUDGET = 5000

print(f'Data directory: {DATA_DIR}')
print(f'Live state dir: {LIVE_DIR}')
print(f'Total budget:   ${TOTAL_BUDGET:,}')
print(f'Date:           {datetime.now().strftime("%Y-%m-%d %H:%M")}')


Data directory: /home/stuar/trade_data/ETFTrader
Live state dir: /home/stuar/code/ETFTrader/fx_signals_live
Total budget:   $5,000
Date:           2026-02-21 16:36


In [2]:
# Cell 1: Check Data Freshness
# ================================

sabr_file = DATA_DIR / 'fx_sabr' / 'sabr_params_historical.parquet'
if sabr_file.exists():
    sabr_raw = pd.read_parquet(sabr_file)
    sabr_raw['date'] = pd.to_datetime(sabr_raw['timestamp']).dt.normalize()
    latest = sabr_raw['date'].max()
    age = (pd.Timestamp.now() - latest).days
    print(f"SABR data latest: {latest.date()} ({age} days ago)")
    if age > 3:
        print(f"  WARNING: Data is {age} days old. Run data collection first!")
    else:
        print("  Data is fresh.")
else:
    print("ERROR: SABR data file not found!")


SABR data latest: 2026-02-19 (2 days ago)
  Data is fresh.


In [3]:
# Cell 2: Detect Active Signals
# =================================

signals = detect_active_signals(DATA_DIR)

if signals:
    print(f"ACTIVE SIGNALS: {len(signals)}")
    print('=' * 60)
    for sig in signals:
        print(f"  {sig['pair']}: {sig['direction'].upper()} signal on {sig['signal_date'].date()}")
        print(f"    Quality: {sig['quality_score']:.3f} ({sig['quality_tercile']})")
        print(f"    Hold: {sig['hold_days']}d")
        if sig['sabr_params']:
            sp = sig['sabr_params']
            print(f"    SABR 1M: F={sp['forward']:.5f}, alpha={sp['alpha']:.4f}, "
                  f"rho={sp['rho']:.3f}, T={sp['T']:.4f}")
else:
    print("No active signals today.")
    print("(Signal fires when fast tenor rho moves while slow tenor is quiet)")


No active signals today.
(Signal fires when fast tenor rho moves while slow tenor is quiet)


In [4]:
# Cell 3: Position Sizing
# ==========================

if signals:
    sized_signals = compute_position_size(signals, total_budget=TOTAL_BUDGET)

    print(f'Position Sizing (budget=${TOTAL_BUDGET:,})')
    print('=' * 60)
    total_margin = 0
    for sig in sized_signals:
        contracts = sig.get('contracts', 0)
        margin = sig.get('margin_estimate', 0)
        total_margin += margin

        if contracts > 0:
            print(f"  {sig['pair']}: {contracts} contracts ({sig['quality_tercile']}), "
                  f"margin ~${margin:,.0f}")
        else:
            print(f"  {sig['pair']}: SKIPPED ({sig.get('skip_reason', 'unknown')})")

    print(f"\nTotal margin: ${total_margin:,.0f} / ${TOTAL_BUDGET:,}")
else:
    sized_signals = []
    print("No signals to size.")


No signals to size.


In [ ]:
# Cell 4: Construct Risk Reversals
# ===================================
# Connects to IB to get CME option chains and snap strikes.

CONFIRM_CONSTRUCT = False  # Set True to connect to IB

if CONFIRM_CONSTRUCT and sized_signals:
    import nest_asyncio
    nest_asyncio.apply()

    for sig in sized_signals:
        if sig.get('contracts', 0) <= 0:
            continue
        rr = construct_risk_reversal(
            ccy=sig['currency'],
            direction=sig['direction'],
            sabr_params=sig['sabr_params'],
        )
        sig['rr'] = rr
        if rr:
            print(f"  {sig['pair']}: Buy {rr['long_contract'].right} @ {rr['long_strike']:.5f}, "
                  f"Sell {rr['short_contract'].right} @ {rr['short_strike']:.5f}, "
                  f"expiry {rr['expiry']}")
        else:
            print(f"  {sig['pair']}: Failed to construct RR")
elif sized_signals:
    print("Set CONFIRM_CONSTRUCT = True to connect to IB and build RR contracts.")
    for sig in sized_signals:
        if sig.get('contracts', 0) > 0 and sig.get('sabr_params'):
            sp = sig['sabr_params']
            K_call, K_put = find_25delta_strikes(sp['forward'], sp['alpha'], sp['T'])
            print(f"  {sig['pair']} (theoretical): 25d call @ {K_call:.5f}, 25d put @ {K_put:.5f}")
else:
    print("No signals to construct.")


No signals to construct.


In [6]:
# Cell 5: Execute Trades
# ========================

CONFIRM_EXECUTE = False  # SET TRUE TO PLACE REAL ORDERS

if CONFIRM_EXECUTE and sized_signals:
    results = execute_fx_signals(sized_signals, LIVE_DIR, confirm=True)
    for r in results:
        print(f"  {r['pair']}: {r['status']}")
elif sized_signals:
    results = execute_fx_signals(sized_signals, LIVE_DIR, confirm=False)
else:
    print("No signals to execute.")


No signals to execute.


In [7]:
# Cell 6: Position Monitoring & Close Expired
# ===============================================

status = check_fx_positions(LIVE_DIR)

if status['n_open'] > 0:
    print(f"Open FX signal positions: {status['n_open']}")
    for pos in status['open_positions']:
        marker = " ** EXPIRED **" if pos['expired'] else ""
        print(f"  {pos['pair']}: {pos['contracts']} contracts ({pos['quality']}), "
              f"{pos['days_left']}d remaining{marker}")

    if status['n_expired'] > 0:
        print(f"\n{status['n_expired']} expired position(s) to close:")
        CONFIRM_CLOSE = False  # Set True to close expired positions
        close_expired_signals(LIVE_DIR, confirm=CONFIRM_CLOSE)
else:
    print("No open FX signal positions.")


No open FX signal positions.
